## Get all the articles to summarize

In [1]:
import requests

host = "http://144.24.201.133:5000"
articles_json = requests.get(f"{host}/articlesToSummarize?skip=1000").json()
print(len(articles_json))

789


## Instanciate BART

In [2]:
from transformers import BartForConditionalGeneration, AutoTokenizer

model_ckpt = "sshleifer/distilbart-cnn-6-6"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = BartForConditionalGeneration.from_pretrained(model_ckpt).cuda()

## Check Cuda comparibility

In [4]:
import torch
from tqdm import tqdm
import math

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# Tokenize and filter by number of pages
If a paper has more than 10 pages, it is filtered out as a single summary will be too long for chat gpt input


In [5]:
page_size = 512

paper_rows = []
skipped_papers = []
for paper in tqdm(articles_json):
    tokens = tokenizer(paper['body'], padding='max_length', return_tensors='pt').to(device)
    n_pages = math.ceil(len(tokens["input_ids"][0])/page_size)
    paper_rows.append(
            {
            "link": paper["link"],
            "body":  paper["body"],
            "summary":  "",
            "input_ids":  tokens
            }
        )

100%|██████████| 789/789 [00:33<00:00, 23.83it/s]


## Summarize Docs
Split tokens in pages of 5 tokens

A single paper (document) is divided in pages (token_splits): each page is an array of tokens.

Documents_tokenized contains the list of papers that must be summarized, and it is an array of documents.

In [6]:
max_size = 512
for row in tqdm(paper_rows):
    n_splits = math.ceil(len(row["input_ids"][0])/max_size)
    document_text_summary = ""
    token_count = 0
    for index in list(range(n_splits)):
        if(index != n_splits-1):
            token_splits = { "input_ids": torch.tensor(row['input_ids']['input_ids'][0][index*max_size:(index+1)*max_size]).unsqueeze(0).to(device),
                                "attention_mask": torch.tensor(row['input_ids']['attention_mask'][0][index*max_size:(index+1)*max_size]).unsqueeze(0).to(device)}

        else:
            token_splits = { "input_ids": torch.tensor(row['input_ids']['input_ids'][0][index*max_size:len(row['input_ids']['input_ids'][0])%max_size + index*max_size]).unsqueeze(0).to(device),
                            "attention_mask": torch.tensor(row['input_ids']['attention_mask'][0][index*max_size:len(row['input_ids']['input_ids'][0])%max_size + index*max_size]).unsqueeze(0).to(device)}

        if(token_splits["input_ids"].nelement() == 0):
          continue
        doc_token_summary = model.generate(input_ids=token_splits['input_ids'],
                            attention_mask=token_splits['attention_mask'],
                            min_length=16,
                            max_length=128)
        token_count += len(doc_token_summary[0])
        extracted_summary = tokenizer.decode(doc_token_summary[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
        
        
        if ("." in extracted_summary):
             document_text_summary += (".".join(extracted_summary.split(".")[0:-1])) + ".\n"
        else:
            document_text_summary += extracted_summary + ".\n"
    requests.post(f"{host}/updateArticle", json={
      "link": row["link"],
      "token_count": token_count,
      "summary": document_text_summary.replace("..", ".")
    }) 


 58%|█████▊    | 460/789 [5:08:24<3:40:35, 40.23s/it]


KeyboardInterrupt: 